<a href="https://colab.research.google.com/github/satishgc227/git-test/blob/master/OTTO_RECOMMENDER_SYSTEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task overview :
The aim of this competition is to predict e-commerce clicks, cart additions, and orders. You'll build a multi-objective recommender system based on previous events in a user session.

Current recommender systems consist of various models with different approaches, ranging from simple matrix factorization to a transformer-type deep neural network. However, no single model exists that can simultaneously optimize multiple objectives. In this competition, you’ll build a single entry to predict click-through, add-to-cart, and conversion rates based on previous same-session events.

Import libraries:

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sys

In [4]:
#!pip install zipfile36

In [5]:
# if sys.version_info >= (3, 6):
#     import zipfile
# else:
#     import zipfile36 as zipfile


# file_name="otto-recommender-system.zip"

In [6]:
# # opening the zip file in READ mode
# with zipfile(file_name, 'r') as zip:
#     # printing all the contents of the zip file
#     zip.printdir()
  
#     # extracting all the files
#     print('Extracting all the files now...')
#     zip.extractall()
#     print('Done!')

In [7]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [8]:
df=pd.read_csv("/content/traian csv.csv")

In [9]:
df

,session,aid,ts,type
0,8000000,766953,1.660420e+12,clicks
1,8000000,1586093,1.660420e+12,clicks
2,8000000,1586093,1.660420e+12,clicks
3,8000000,682970,1.660420e+12,clicks
4,8000000,799794,1.660420e+12,clicks
...,...,...,...,...
535058,8049857,520461,1.660430e+12,clicks
535059,8049857,383482,1.660430e+12,clicks
535060,8049858,1127009,1.660430e+12,clicks
535061,8049858,35971,1.660930e+12,clicks


Data preprocessing:

In [10]:
df.isnull().sum()

session    0
aid        0
ts         1
type       1
dtype: int64

In [11]:
df.dropna()

,session,aid,ts,type
0,8000000,766953,1.660420e+12,clicks
1,8000000,1586093,1.660420e+12,clicks
2,8000000,1586093,1.660420e+12,clicks
3,8000000,682970,1.660420e+12,clicks
4,8000000,799794,1.660420e+12,clicks
...,...,...,...,...
535057,8049856,1440375,1.660430e+12,clicks
535058,8049857,520461,1.660430e+12,clicks
535059,8049857,383482,1.660430e+12,clicks
535060,8049858,1127009,1.660430e+12,clicks


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 535063 entries, 0 to 535062
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   session  535063 non-null  int64  
 1   aid      535063 non-null  int64  
 2   ts       535062 non-null  float64
 3   type     535062 non-null  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 16.3+ MB


In [13]:
# import json
# f=open("/content/test.jsonl")
# chunks=f.read()
# chunks

In [14]:
#print(chunks)

In [15]:
# import pandas as pd
# import json
# import glob
# from pandas.io.json import json_normalize

# json_files = glob.glob("*.json")
# dfs = []
# for file in json_files:
#     with open("/content/test.jsonl") as f:
#         for line in f.readlines():
#             df = pd.json_normalize(json.loads(line))
#             list_= ['','Item.metadata.M.timestamp.S','Item.sensor.M.celcius.N','Item.sensor.M.water.N']
#             df = df.loc[:, df.columns.isin(list_)]
#             dfs.append(df)
# df_combine = pd.concat(dfs, sort=False)
# df_combine.to_csv('json_to_raw.csv',index= None)

In [16]:
# import zipfile
# from google.colab import drive

# drive.mount('/content/drive/')

# zip_ref = zipfile.ZipFile("/content/drive/Othercomputers/My Laptop/Downloads/train.jsonl.zip", 'r')
# zip_ref.extractall("/content/drive/Othercomputers/My Laptop/Downloads/tmp")
# zip_ref.close()

In [17]:
# import json
# dd=open("/content/drive/Othercomputers/My Laptop/Downloads/tmp/train.jsonl")

In [18]:
# import pandas as pd
# chunks=pd.read_json(dd,lines=True, chunksize=1000)

In [19]:
#fg=pd.DataFrame(chunks)

In [21]:
df.shape

(535063, 4)

In [25]:
df['session'].unique()


array([8000000, 8000001, 8000002, ..., 8049857, 8049858, 8049859])

In [27]:
df.session.unique().shape

(49860,)

In [33]:
s=df.groupby('session')['aid','ts','type'].count()
print(s)

         aid   ts  type
session                
8000000    6    6     6
8000001    2    2     2
8000002    7    7     7
8000003    5    5     5
8000004  151  151   151
...      ...  ...   ...
8049855   11   11    11
8049856    2    2     2
8049857    2    2     2
8049858    2    2     2
8049859    1    0     0

[49860 rows x 3 columns]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [35]:
df.ts

0         1.660420e+12
1         1.660420e+12
2         1.660420e+12
3         1.660420e+12
4         1.660420e+12
              ...     
535058    1.660430e+12
535059    1.660430e+12
535060    1.660430e+12
535061    1.660930e+12
535062             NaN
Name: ts, Length: 535063, dtype: float64

In [58]:
import datetime
d=df['ts'].max()
f=df['ts'].min()
f

1660420000000.0

In [50]:
datetime.datetime.fromtimestamp(f/1000),datetime.datetime.fromtimestamp(d/1000)

(datetime.datetime(2022, 8, 13, 19, 46, 40),
 datetime.datetime(2022, 8, 28, 20, 53, 20))

In [56]:
df['ts'].values

array([1.66042e+12, 1.66042e+12, 1.66042e+12, ..., 1.66043e+12,
       1.66093e+12,         nan])

In [61]:
df['ts'].values[0]

1660420000000.0

In [66]:
session_duration = df.groupby('session').apply(lambda x: (x['ts'].values[-1] - x['ts'].values[0])/(1000*60*60))
session_duration.max()

361.1111111111111

Let's convert timestamp to the readable time:

In [72]:
df['ts']=pd.to_datetime(df['ts'])

In [77]:
b=df['ts']

In [81]:
print(f'Years:{b.dt.year.unique()} , \n Months"{b.dt.month.unique()}, \n Days:{b.dt.day.unique()} ')

Years:[1970.   nan] , 
 Months"[ 1. nan], 
 Days:[ 1. nan] 
